# EEG analysis - part 3

In [60]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, svm
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.feature_selection import chi2, mutual_info_classif, f_classif
import os
import Queue
import sys
import tarfile
import gzip
import pandas
import random
from pyunicorn.timeseries.recurrence_plot import RecurrencePlot
import nolds
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression

from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import warnings
warnings.filterwarnings("ignore") #, category=DeprecationWarning) 

# %matplotlib inline

## Dataset retrieval

In [45]:
with open('dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)
    
with open('labels.pickle', 'rb') as f:
    labels = pickle.load(f)

In [46]:
def split(dataset, *splits):
    train_dataset, train_labels = dataset[:splits[0]], labels[:splits[0]]
    test_dataset, test_labels = dataset[splits[0]:splits[1]], labels[splits[0]:splits[1]]
    valid_dataset, valid_labels = dataset[splits[1]:], labels[splits[1]:]
    
    return train_dataset, train_labels, test_dataset, test_labels, valid_dataset, valid_labels
    
train_dataset, train_labels, test_dataset, test_labels, valid_dataset, valid_labels = split(dataset, 1000, 1100)

In [47]:
# Validate we have balanced datasets
np.histogram(train_labels, bins=2), np.histogram(test_labels, bins=2), np.histogram(valid_labels, bins=2)

((array([499, 501]), array([0. , 0.5, 1. ])),
 (array([58, 42]), array([0. , 0.5, 1. ])),
 (array([43, 57]), array([0. , 0.5, 1. ])))

In [61]:
train_labels = to_categorical(train_labels, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)
valid_labels = to_categorical(valid_labels, num_classes=2)

## Training

In [83]:
image_size = 256
num_channels = 5
batch_size = 25
epochs=25

In [84]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(image_size, image_size, num_channels), batch_size=batch_size, data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

In [85]:
model = define_model()
model.fit(
        train_dataset, train_labels,
        batch_size=batch_size,
#         steps_per_epoch=2000 // batch_size,
        epochs=epochs,
#         validation_data=(valid_dataset, valid_labels),
#         validation_steps=800 // batch_size
        )

print(model.evaluate(test_dataset, test_labels, batch_size=batch_size))
# model.save_weights('.h5') 

Epoch 1/25
1000/1000 [==============================] - 10s 10ms/step - loss: 7.6396 - acc: 0.5010
Epoch 2/25
1000/1000 [==============================] - 9s 9ms/step - loss: 3.6462 - acc: 0.4910
Epoch 3/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.7203 - acc: 0.5510
Epoch 4/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.7233 - acc: 0.5685
Epoch 5/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.7406 - acc: 0.6305
Epoch 6/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.6718 - acc: 0.6880
Epoch 7/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.6044 - acc: 0.7360
Epoch 8/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.5075 - acc: 0.7770
Epoch 9/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4175 - acc: 0.8300
Epoch 10/25
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4268 - acc: 0.8500
Epoch 11/25
1000/1000 [====

With some tweaking, we have almost 73% accuracy - not breathtaking, but not bad, either, given we expected little to no relationship in this data at all! (This is just a toy dataset - the orignal experiment was designed to expose a different relationship.)

Let's try to find the best parameters for it with grid search, just for fun!